In [33]:
import numpy as np
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.viz.bokeh.DiffusionTrend import DiffusionTrend
from ndlib.viz.bokeh.DiffusionPrevalence import DiffusionPrevalence
from bokeh.io  import output_notebook, show
from bokeh.plotting import figure, from_networkx

output_notebook()

Loading BokehJS ...

In [34]:
# Seed
rnd_seed = 42

# Network topology
g = nx.erdos_renyi_graph(1000, 0.1, seed=rnd_seed)

# Model selection
model = ep.SEIRModel(g)

# Model iteration
t_steps = 400

In [35]:
# Model Configuration
cfg = mc.Configuration()
cfg.add_model_parameter('beta', 0.01)
cfg.add_model_parameter('gamma', 0.005)
cfg.add_model_parameter('alpha', 0.05)
cfg.add_model_parameter("fraction_infected", 0.05)
model.set_initial_status(cfg)

# Simulation execution
iterations = model.iteration_bunch(t_steps)
trends = model.build_trends(iterations)

In [61]:
list(iterations[0]['status'].values()).count(0)

950

In [49]:
print(np.sum(list(iterations[0]['status'].values())) ==1)

False


In [13]:
# Visualization
viz = DiffusionTrend(model, trends)
p = viz.plot(width=800, height=600)
show(p)

In [14]:
# Visualization prevalence
viz = DiffusionPrevalence(model, trends)
p = viz.plot(width=800, height=600)
show(p)

## A custom model

In [112]:
import ndlib.models.CompositeModel as gc
import ndlib.models.ModelConfig as mc
import ndlib.models.compartments.NodeStochastic as NodeStochastic
import ndlib.models.compartments.NodeCategoricalAttribute as NodeCategoricalAttribute
import ndlib.models.compartments.NodeNumericalAttribute as NodeNumericalAttribute
import ndlib.models.compartments.NodeThreshold as NodeThreshold
import ndlib.models.compartments.CountDown as CountDown
import ndlib.models.compartments.ConditionalComposition as ConditionalComposition

In [249]:
# Disease parameters
population_size = 1000
t_steps = 60
days_recovery_infected = 14
days_recovery_critical = 21
beds = 5.98 * population_size / 1000

proba_exposed = 0.1
# proba_asymptomic = 0.2
proba_critical = 0.167
proba_reinfection = 0.0025

case_fatality_rate = 0.03
basal_contagion_rate = 0.2

In [250]:
def id_dynamic_rule(model, s_0, s_1, rule):
    model.add_rule(s_0, s_1, rule)
    return model.compartment_progressive - 1

def change_rule(model, rule_id, new_rule):
    s_0 = model.compartment[rule_id][0]
    s_1 = model.compartment[rule_id][1]
    model.compartment[rule_id] = (s_0, s_1, new_rule)
    
def update_on_beds(model, status, id_bedsrule):
    
    crit_id = model.get_status_map()["Critical"]
    current_critical = list(status.values()).count(crit_id)
    
    if current_critical >= beds:
        new_rule = NodeStochastic(2 * case_fatality_rate)
        change_rule(model, id_bedsrule, new_rule)
    else:
        new_rule = NodeStochastic(case_fatality_rate)
        change_rule(model, id_bedsrule, new_rule)
    

In [251]:
# Starting network
g = nx.erdos_renyi_graph(population_size, 0.15)

true_event = NodeStochastic(1.0)

# Setting node attribute
attr = {n: {
    "Sex": np.random.choice(['male', 'female']),
    "Age": np.random.choice(range(0, 100)),
    "p_symptoms": proba_asymptomic} for n in g.nodes()}
nx.set_node_attributes(g, attr)

# Composite Model instantiation
model = gc.CompositeModel(g)

# Model statuses
model.add_status("Susceptible")
model.add_status("Exposed")
model.add_status("Infected")
model.add_status("Critical")
model.add_status("Recovered")
model.add_status("Dead")

# Contagion rules
# - Proba of becoming critical
model.add_rule("Infected", "Critical", NodeStochastic(proba_critical))

# - Recovery rules: time period or proba
rec1 = CountDown("recovery", iterations=days_recovery_infected)
rec2 = CountDown("recovery_critical", iterations=days_recovery_critical)

base_recovery = ConditionalComposition(NodeStochastic(0.15), true_event, rec1)
critical_recovery = ConditionalComposition(NodeStochastic(0.05), true_event, rec2)

model.add_rule("Infected", "Recovered", base_recovery)
model.add_rule("Critical", "Recovered", critical_recovery)

# - Dead proba
# model.add_rule("Infected", "Dead", NodeStochastic(case_fatality_rate))

crit_dead_id = id_dynamic_rule(model, "Critical", "Dead", NodeStochastic(case_fatality_rate))
# - Basal contagion
model.add_rule("Susceptible", "Exposed", NodeStochastic(proba_exposed))
model.add_rule("Exposed", "Susceptible", NodeStochastic(1.1 * proba_exposed))
model.add_rule("Exposed", "Infected", NodeStochastic(basal_contagion_rate, triggering_status="Infected"))
model.add_rule("Exposed", "Infected", NodeStochastic(basal_contagion_rate, triggering_status="Critical"))

model.add_rule("Recovered", "Exposed", NodeStochastic(proba_reinfection))





In [252]:
# Model initial status configuration
config = mc.Configuration()
config.add_model_parameter('fraction_infected', 0.05)

# Set initial state
model.set_initial_status(config)



In [253]:
# Simulate updating on critical patients vs available beds:
iterations = []
for it in range(0, t_steps):
    its = model.iteration(node_status=True)
    status_t = its['status']
    update_on_beds(model, status_t, crit_dead_id)
    
    iterations.append(its)

In [254]:
# Build trends
# iterations = model.iteration_bunch(t_steps)
trends = model.build_trends(iterations)

In [255]:
# Visualization
viz = DiffusionTrend(model, trends)
p = viz.plot(width=800, height=600)
show(p)

In [256]:
# Visualization prevalence
viz = DiffusionPrevalence(model, trends)
p = viz.plot(width=800, height=600)
show(p)